In [12]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential 
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from skimage.transform import resize
import h5py
import os

In [13]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Resize images to 42x42
x_train_resized = np.zeros((x_train.shape[0], 42, 42))
for i in range(x_train.shape[0]):
    x_train_resized[i] = resize(x_train[i], (42, 42))

x_test_resized = np.zeros((x_test.shape[0], 42, 42))
for i in range(x_test.shape[0]):
    x_test_resized[i] = resize(x_test[i], (42, 42))

# Reshape to add channel dimension (required for Keras)
x_train_resized = x_train_resized.reshape(-1, 42, 42, 1)
x_test_resized = x_test_resized.reshape(-1, 42, 42, 1)

# Normalize pixel values to range [0, 1]
x_train_resized = x_train_resized.astype('float32') / 255.0
x_test_resized = x_test_resized.astype('float32') / 255.0

# One-hot encode labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [14]:
model = Sequential()
model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(4, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
# model.fit(x_train_resized, y_train, batch_size=128, epochs=15, validation_data=(x_test_resized, y_test))

# Quantize the weights and biases
model_quantized = Sequential()
for layer in model.layers:
    if isinstance(layer, Dense) or isinstance(layer, Conv2D):
        weights, biases = layer.get_weights()
        weights_quantized = np.round(weights * 4) / 4  # Quantize weights to 2 bits for fixed point part
        biases_quantized = np.round(biases * 4) / 4     # Quantize biases to 2 bits for fixed point part
        layer.set_weights([weights_quantized, biases_quantized])
    model_quantized.add(layer)

model_quantized.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



c:\Users\Rishit Thakkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [22]:
# model_quantized.save('quantized_mnist_model.h5')
model_quantized = load_model('quantized_mnist_model.h5')

loss, accuracy = model_quantized.evaluate(x_test_resized, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 801us/step - accuracy: 0.8561 - loss: 0.6173
Test Loss: 0.5866955518722534
Test Accuracy: 0.8644999861717224


In [23]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Conv2D
from keras.preprocessing import image

# Define the model architecture
model = Sequential()
model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))  

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Quantize the weights and biases
model_quantized = Sequential()
for layer in model.layers:
    if isinstance(layer, Conv2D):
        weights, biases = layer.get_weights()
        weights_quantized = np.round(weights * 4) / 4  # Quantize weights
        biases_quantized = np.round(biases * 4) / 4     # Quantize biases
        layer.set_weights([weights_quantized, biases_quantized])
    model_quantized.add(layer)

model_quantized(tf.keras.Input((42, 42, 1)))  
model_quantized.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load and preprocess the input image
input_img_path = 'random_resized_image.png'
img = image.load_img(input_img_path, target_size=(42, 42), color_mode='grayscale')  
input_img_data = image.img_to_array(img)
input_img_data = np.expand_dims(input_img_data, axis=0)

# Predict the output of the first convolutional layer
conv1_layer_output = Model(inputs=model_quantized.input, outputs=model_quantized.output)
conv1_output = conv1_layer_output.predict(input_img_data)

print("Output shape of the first convolutional layer:", conv1_output.shape)

# Save the convolutional layer output
with open('Python_prog_out_conv.txt', 'w') as file:
    for c in range(conv1_output.shape[3]):  # Iterate over channels
        file.write("Channel %d:\n" % (c + 1))
        for i in range(conv1_output.shape[1]):  # Iterate over height
            file.write("[ ")
            for j in range(conv1_output.shape[2]):  # Iterate over width
                file.write("%.2f" % conv1_output[0, i, j, c])
                if j != conv1_output.shape[2] - 1:
                    file.write(", ")
            file.write(" ]\n")
        file.write("\n")

print("Convolutional layer output saved")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Output shape of the first convolutional layer: (1, 40, 40, 4)
Convolutional layer output saved
